In [1]:
import sqlite3

In [25]:
'''class crawler:
    def __init__(self , dbname):
        #数据库资源对象
        self.con = sqlite3.connect(dbname)
    
    def __del__(self):
        self.con.close()
    
    def dbcommit(self):
        self.con.commit()
    
    
    def getentryid(self , table , field , value , createnew = True):
        cur = self.con.execute(
        "select rowid from %s where %s = '%s'" % (table , field , value))
        
        res = cur.fetchone()
        
        if res == None:
            cur = self.con.execute(
            "insert into %s (%s) values ('%s')" % (table , field , value))
            
            return cur.lastrowid
        else:
            return res[0]
        

    def addtoindex(self , url , soup):
        if self.isindexed(url):
            return
        print('indexing ' + url)
        
        text = self.gettextonly(soup)
        words = self.separatewords(text)

        urlid = self.getentryid('urllist' , 'url' , 'url')
        
        for i in range(len(words)):
            word = words[i]
            
            if word in ignorewords:
                continue
            
            worldid = self.getentryid('wordlist' , 'word' , word)
            self.con.execute('insert into wordlocation(urlid , wordid , location) \
            values (%d,%d,%d)' %(urlid , wordid , i))
        
        
    def gettextonly(self , soup):
        v = soup.string
        
        if v == None:
            c = soup.contents
            resulttext = ''
            
            for t in c:
                subtext = self.gettextonly(t)
                resulttext += subtext+'\n'
                
            return resulttext
        else:
            return v.strip()
    
    def separatewords(self , text):
        splitter = re.compile('\\W*')
        
        return [s.lower() for s in splitter.split(text) if s != '']
    
    def isindexed(self , url):
        u = self.con.execute(
        "select rowid from urllist where url='%s'" % url).fetchone()
        
        if u!=None:
            v = self.con.execute(
            "select * from wordlocation where urlid=%d" % u[0]).fetchone()
        if v!= None:
            return True
        
        return False
    
    def addlinkref(self , urlFrom , urlTo , linkText):
        pass
    
    def crawl(self , pages , depth = 1):
        
        for i in range(depth):
            newpages = set()
            
            for page in pages:
                try:
                    c = urllib2.urlopen(page)
                except:
                    print('could not open %s' % page)
                    continue
                soup = BeautifulSoup(c.read())
                
                self.addtoindex(page , soup)
                
                links = soup('a')
                
                for link in links:
                    if ('href' in dict(link.attrs)):
                        url = urljoin(page , link['href'])
                        
                        if url.find("'") != -1:
                            continue
                        url = url.split('#')[0]
                        
                        if url[0:4] == 'http' and not self.isindexed(url):
                            newpages.add(url)
                        
                        linkText = self.gettestonly(link)
                        
                        self.addlinkref(page , url , linkText)
                
                self.dbcommit()
            pages = newpages
    
    def createindextables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid , worldid , location)')
        self.con.execute('create table link(fromid integer , toid integer)')
        self.con.execute('create table linkwords(wordid , linkid)')
        
        self.con.execute('create index wordidxon wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        
        self.dbcommit()
    

    
    def createindextables(self):
        pass
'''

In [30]:
class crawler:
    def __init__(self,dbname):
        self.con=sqlite3.connect(dbname)
        #连接并建立数据库, dbname 随意, 'xxx.db'就可以
    def __del__(self):
        self.con.close()
    def dbcommit(self):
        self.con.commit()
    
    def getentryid(self,table,field,value,createnew=True):
        cur=self.con.execute(
            "select rowid from %s where %s='%s'" %(table,field,value))
        res=cur.fetchone()
        if res==None:
            cur=self.con.execute(
                "insert into %s (%s) values ('%s')" % (table,field,value))
            return cur.lastrowid
        else:
            return res[0]
    
    
    def addtoindex(self,url,soup):
        if self.isindexed(url): return
        print 'Indexing',url
        
        #Get words
        text=self.gettextonly(soup)
        words=self.separatewords(text)
        
        #Get URL id
        urlid=self.getentryid('urllist','url',url)
        
        # Link word to url
        for i in range(len(words)):
            word=words[i]
            if word in ignorewords: continue
            wordid=self.getentryid('wordlist','word',word)
            self.con.execute("insert into wordlocation(urlid,wordid,location) \
            values(%d,%d,%d)" % (urlid,wordid,i))
            
            
    
    def gettextonly(self,soup):
        v=soup.string
        if v==None:
            c=soup.contents
            resulttext=''
            for t in c:
                subtext=self.gettextonly(t)
                resulttext+=subtext+'\n'
            return resulttext
        else:
            return v.strip()
    
    def separatewords(self,text):
        splitter=re.compile('\\W*')
        return [s.lower() for s in splitter.split(text) if s!='']
    
    def isindexed(self,url):
        u=self.con.execute(
            "select rowid from urllist where url='%s'" % url).fetchone()
        if u!=None:
            #if crawled
            v=self.con.execute(
                'select * from wordlocation where urlid=%d' % u[0]).fetchone()
            if v != None: return True
        return False
    
    def addlinkref(self,urlFrom,urlTo,linkText):
        pass
    
    def crawl(self,pages,depth=2):
        for i in range(depth):
            newpages=set()
            for page in pages:
                try:
                    c=urllib2.urlopen(page)
                except:
                    print "Could not open",page
                    continue
                soup=BeautifulSoup(c.read())
                self.addtoindex(page,soup)
                
                links=soup('a')
                for link in links:
                    if 'href' in dict(link.attrs):
                        url=urljoin(page,link['href'])
                        if url.find("'") != -1:
                            continue
                        url=url.split('#')[0] #remove location portion
                        if url[0:4]=='http' and not self.isindexed(url):
                            newpages.add(url)
                            linkText=self.gettextonly(link)
                            self.addlinkref(page,url,linkText)
                self.dbcommit()
            pages=newpages
            
    def createindextables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.dbcommit()

    #pagerank而生
    def calculatepagerank(self , iterations = 20):
        self.con.execute('drop table if exists pagerank')
        self.con.execute('create table pagerank(urlid primary key , score)')
        
        self.con.execute('insert into pagerank select rowid , 1.0 from urllist')
        self.dbcommit()
        
        for i in range(iterations):
            print('iteration %d' % (i))
            
            for (urlid,) in self.con.execute('select rowid from urllist'):
                pr = 0.15
                
                for (linker , ) in self.con.execute(
                "select distinct fromid from link where toid=%d" % urlid):
                    linkingpr = self.con.execute(
                    "select score from pagerank where urlid=%d" % linker).fetchone()[0]
                    
                    linkingcount = self.con.execute(
                    "select count(*) from link where fromid = %d" % linker).fetchone()[0]
                    
                    pr += 0.85*(linkingpr / linkingcount)
                self.con.execute('update pagerank set score=%f where urlid =%d' % (pr , urlid))
        
        self.dbcommit()

In [3]:
import urllib2

In [3]:
c = urllib2.urlopen('http://www.4399.com')
#c = urllib2.urlopen('http://kiwitobes.com/wiki/Programming_language.html')

contents = c.read()

print(contents[0:50])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Trans


In [4]:
from BeautifulSoup import *
from urlparse import urljoin

In [5]:
ignorewords = set(['the' , 'of','to' ,'and' , 'a' , 'in','is','it'])

In [6]:
pagelist = ['http://www.gamersky.com/']
crawler_ = crawler('searchindex.db')
crawler_.createindextables()
crawler_.crawl(pagelist)

Indexing http://www.gamersky.com/
Indexing http://dj.changyou.com/
Indexing http://www.gamersky.com/review/
Indexing http://www.gamersky.com/z/re4/
Indexing http://www.gamersky.com/z/madmax/
Indexing http://www.gamersky.com/z/maxpayne3/
Indexing http://www.gamersky.com/news/201803/1030459.shtml
Indexing http://www.gamersky.com/z/sleepingdogs/
Indexing http://www.gamersky.com/news/pc/zx/
Indexing http://www.gamersky.com/tech/201803/1031393.shtml
Indexing http://www.gamersky.com/z/endlessspace2/
Indexing http://www.gamersky.com/z/amazingspiderman/
Indexing http://www.gamersky.com/z/dragonballx2/
Indexing http://www.gamersky.com/z/halflife2/
Indexing http://www.gamersky.com/z/pal2/
Indexing http://www.gamersky.com/ent/201803/1031378.shtml
Indexing http://www.gamersky.com/z/oatbf/
Indexing http://www.gamersky.com/news/201803/1031383.shtml
Indexing http://www.gamersky.com/z/shio/
Indexing http://www.gamersky.com/tech/201804/1031568.shtml
Indexing http://ku.gamersky.com/2017/pal7/
Indexing h

Indexing http://www.gamersky.com/zl/academy/
Indexing http://www.gamersky.com/z/galciv3/
Indexing http://www.gamersky.com/z/tes6/
Indexing http://www.gamersky.com/wenku/201803/1031126.shtml
Indexing http://www.gamersky.com/z/geteven/
Indexing http://www.gamersky.com/ent/201803/1029284.shtml
Indexing http://shouyou.gamersky.com/
Indexing http://www.gamersky.com/z/frostpunk/
Indexing http://ku.gamersky.com/2017/naruto-to-boruto-shinobi-striker/
Indexing http://shouyou.gamersky.com/ku/850299.shtml
Indexing http://www.5068.com/
Indexing http://www.gamersky.com/wenku/201803/1030673.shtml
Indexing http://www.gamersky.com/z/deadrising/
Indexing http://www.gamersky.com/z/childoflight/
Indexing http://www.gamersky.com/z/pvz2/
Indexing http://www.gamersky.com/z/aragami/
Indexing http://www.gamersky.com/z/moh/
Indexing http://donghua.gamersky.com/v/961590.shtml
Indexing http://www.gamersky.com/z/destiny2/
Indexing http://www.gamersky.com/ent/201803/1029507.shtml
Indexing http://www.gamersky.com/z

Indexing http://www.gamersky.com/z/rsor/
Indexing http://www.gamersky.com/z/legobatman3/
Indexing http://www.gamersky.com/z/rome2/
Indexing http://www.gamersky.com/z/re5/
Indexing http://www.gamersky.com/news/201803/1028849.shtml
Indexing http://shouyou.gamersky.com/news/201803/1031419.shtml
Indexing http://shouyou.gamersky.com/z/gundambattle/
Indexing http://www.gamersky.com/z/ninokuni2/
Indexing http://www.gamersky.com/top/
Indexing http://down.gamersky.com/pc/201803/1030341.shtml
Indexing http://www.gamersky.com/z/bulletstorm/
Indexing http://down.gamersky.com/pc/201803/1023568.shtml
Indexing http://www.gamersky.com/wenku/201803/1030791.shtml
Indexing http://donghua.gamersky.com/v/955921.shtml
Indexing http://down.gamersky.com/
Indexing http://www.gamersky.com/ent/201802/1011073.shtml
Indexing http://www.gamersky.com/review/201803/1031473.shtml
Indexing http://www.gamersky.com/z/anthem/
Indexing http://www.gamersky.com/handbook/
Indexing http://donghua.gamersky.com/v/961823.shtml
In

Indexing http://www.gamersky.com/z/caseanimatronics/
Indexing http://shouyou.gamersky.com/news/201803/1030958.shtml
Indexing http://www.gamersky.com/handbook/201803/1031189.shtml
Indexing http://www.gamersky.com/z/thecoma/
Indexing http://www.gamersky.com/ent/201804/1031616.shtml
Indexing http://www.gamersky.com/review/ol/
Indexing http://www.gamersky.com/z/stickoftruth/
Indexing http://www.gamersky.com/z/pal7/
Indexing http://down.gamersky.com/pc/201803/1028602.shtml
Indexing http://www.gamersky.com/zt/crysis/
Indexing http://www.gamersky.com/z/tes5/
Indexing http://ku.gamersky.com/2017/shadow-of-the-tomb-raider/
Indexing http://www.gamersky.com/z/oneuponlight/
Indexing http://www.gamersky.com/z/f13game/
Indexing http://www.gamersky.com/z/watchdogs2/
Indexing http://www.gamersky.com/z/observer/
Indexing http://iask.sina.com.cn/
Indexing http://shouyou.gamersky.com/ku/
Indexing http://www.gamersky.com/z/totalwarattila/
Indexing http://ku.gamersky.com/2014/Mount-Blade-II-Bannerlord/
Ind

Indexing http://i.gamersky.com/u/229782
Indexing http://www.265g.com/
Indexing http://shouyou.gamersky.com/z/cook/
Indexing http://www.gamersky.com/news/201804/1031626.shtml
Indexing http://www.gamersky.com/z/orientalempires/
Indexing http://ol.gamersky.com/z/qhyx/
Indexing http://www.gamersky.com/z/grandagesmedieval/
Indexing http://www.gamersky.com/z/tdu2/
Indexing http://www.gamersky.com/z/shadowtactics/
Indexing http://www.gamersky.com/z/overcooked/
Indexing http://www.gamersky.com/z/kuniokun/
Indexing http://www.gamersky.com/z/evolve/
Indexing http://www.gamersky.com/z/nfs19/
Indexing http://ku.gamersky.com/
Indexing http://www.gamersky.com/z/metroexodus/
Indexing http://www.gamersky.com/news/201804/1031555.shtml
Indexing http://www.gamersky.com/z/stalkercop/
Indexing http://www.gamersky.com/ent/201803/1028823.shtml
Indexing http://donghua.gamersky.com/v/803722.shtml
Indexing http://www.gamersky.com/z/AlanWake/
Indexing http://www.gamersky.com/z/aots/
Indexing http://shouyou.gamer

ValueError: Invalid IPv6 URL

In [31]:
crawler_ = crawler('searchindex.db')
crawler_.calculatepagerank()
#对PageRank分值进行迭代计算

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19


In [41]:
cur = crawler_.con.execute('select * from pagerank order by score desc')
for i in range(20):
    print(cur.next())

(1, 0.15)
(2, 0.15)
(3, 0.15)
(4, 0.15)
(5, 0.15)
(6, 0.15)
(7, 0.15)
(8, 0.15)
(9, 0.15)
(10, 0.15)
(11, 0.15)
(12, 0.15)
(13, 0.15)
(14, 0.15)
(15, 0.15)
(16, 0.15)
(17, 0.15)
(18, 0.15)
(19, 0.15)
(20, 0.15)


In [40]:
for i in range(5):
    print(e.geturlname(i+1))

http://www.gamersky.com/
http://dj.changyou.com/
http://www.gamersky.com/review/
http://www.gamersky.com/z/re4/
http://www.gamersky.com/z/madmax/


In [8]:
[row for row in crawler_.con.execute(
"select rowid from wordlocation where wordid = 1")]

[(1,),
 (1852,),
 (2218,),
 (3210,),
 (3923,),
 (4492,),
 (5063,),
 (5566,),
 (6215,),
 (6611,),
 (7156,),
 (7612,),
 (8091,),
 (8829,),
 (9302,),
 (9717,),
 (10165,),
 (10641,),
 (11134,),
 (11544,),
 (12057,),
 (12301,),
 (12689,),
 (12974,),
 (13634,),
 (14119,),
 (14584,),
 (15097,),
 (15618,),
 (16066,),
 (16586,),
 (16897,),
 (17470,),
 (18032,),
 (18543,),
 (18925,),
 (19393,),
 (21903,),
 (22401,),
 (22811,),
 (23537,),
 (23988,),
 (24669,),
 (25323,),
 (26200,),
 (26512,),
 (27558,),
 (34647,),
 (35213,),
 (35777,),
 (36294,),
 (39229,),
 (39492,),
 (39948,),
 (40396,),
 (40823,),
 (41275,),
 (41694,),
 (42267,),
 (42769,),
 (43144,),
 (43522,),
 (43898,),
 (44740,),
 (45457,),
 (45895,),
 (46298,),
 (46714,),
 (47263,),
 (47548,),
 (47968,),
 (48627,),
 (49036,),
 (49487,),
 (50311,),
 (50913,),
 (51113,),
 (51610,),
 (52031,),
 (52595,),
 (66234,),
 (66662,),
 (71164,),
 (71774,),
 (72039,),
 (72527,),
 (72985,),
 (73918,),
 (74288,),
 (75093,),
 (75800,),
 (76816,),
 (77278

In [50]:
#接下来是搜索lei
class searcher:
    def __init__(self , dbname):
        self.con = sqlite3.connect(dbname)
    
    def __del__(self):
        self.con.close()
        
    def getmatchrows(self , q):
        fieldlist = 'w0.urlid'
        tablelist = ''
        clauselist = ''
        
        wordids = []
        
        words = q.split(' ')
        tablenumber = 0
        
        for word in words:
            wordrow = self.con.execute(
            "select rowid from wordlist where word = '%s'" % word).fetchone()
            
            if wordrow != None:
                wordid = wordrow[0]
                wordids.append(wordid)
                
                if tablenumber>0:
                    tablelist += ','
                    clauselist += ' and '
                    clauselist += 'w%d.urlid=w%d.urlid and '%(tablenumber-1 , tablenumber)
                
                fieldlist += ',w%d.location' % tablenumber
                tablelist += 'wordlocation w%d' % tablenumber
                clauselist += 'w%d.wordid=%d' % (tablenumber , wordid)
                
                tablenumber += 1
        
        fullquery = 'select %s from %s where %s' % (fieldlist , tablelist , clauselist)
        cur = self.con.execute(fullquery)
        
        rows = [row for row in cur]
        
        return rows , wordids
    
    def getscoredlist(self , rows , wordids):
        totalscores = dict([(row[0] , 0) for row in rows])
        
        #这里需要对不同的度量方法进行更新 也可以进行加权进行
        
        #weights = [(1.0 , self.frequencyscore(rows))] #
        
        weights = [(1.0 , self.locationscore(rows)),
                   (1.0 , self.frequencyscore(rows)),
                   (1.0 , self.pagerankscore(rows)),
                   (1.0 , self.linktextscore(rows , wordids))]
        
        for (weight , scores) in weights:
            for url in totalscores:
                totalscores[url] += weight*scores[url]
        
        return totalscores
    
    def geturlname(self , id_):
        return self.con.execute(
        "select url from urllist where rowid=%d" % id_).fetchone()[0]
    
    def query(self , q):
        rows , wordids = self.getmatchrows(q)
        scores = self.getscoredlist(rows , wordids)
        rankedscores = sorted([(score , url) for (url , score) in scores.items()] ,
                             reverse = 1)
        
        for (score , urlid) in rankedscores[0:10]:
            print('%f\t%s' % (score , self.geturlname(urlid)))
    
    #对评分归一化函数
    def normalizescores(self , scores , smallIsBetter = 0):
        vsmall = 0.00001
        
        if smallIsBetter:
            minscore = min(scores.values())
            return dict([(u , float(minscore)/max(vsmall , l)) 
                        for (u , l) in scores.items()])
        else:
            maxscore = max(scores.values())
            if maxscore == 0:
                maxscore = vsmall
            
            return dict([(u , float(c)/maxscore) 
                        for (u , c) in scores.items()])
    
    #单词频度统计函数
    def frequencyscore(self , rows):
        counts = dict([(row[0] , 0) for row in rows])
        
        for row in rows:
            counts[row[0]] += 1
        
        return self.normalizescores(counts)
    
    #根据搜索的单词的位置进行评分
    #如果要使用这种评分方式 就把getscoredlist函数中 weights=--- 这一行变为
    #weights = [(1.0 , self.locationscore(rows))]
    def locationscore(self , rows):
        locations = dict([(row[0] , 1000000) 
                         for row in rows])
        
        for row in rows:
            loc = sum(row[1:])
            
            if loc<locations[row[0]]:
                locations[row[0]] = loc
        
        return self.normalizescores(locations , smallIsBetter = 1)
    
    #rows中存放是单词的位置
    #如同上一种度量方式
    def distancescore(self , rows):
        if len(rows[0])<=2:
            return dict([(row[0] , 1.0) for row in rows])
        
        mindistance = dict([(row[0] , 1000000) 
                           for row in rows])
        
        for row in rows:
            dist = sum([abs(row[i]-row[i-1]) 
                       for i in range(2 , len(row))])
            
            if dist<mindistance[row[0]]:
                mindistance[row[0]] = dist
        
        return self.normalizescores(mindistance , smallIsBetter = 1)
    
    #使用外部的回指链接
    #就是别的网站对这个网站的链接的引用 作为网站的衡量方式
    
    #简单计数的方式
    def inboundlinkscore(self , rows):
        uniqueurls = set([row[0] for row in rows])
        iboundcount = dict([(u , self.con.execute(
        "select count(*) from link where toid = %d" % u).fetchone()[0]) 
                           for u in uniqueurls])
        
        return self.normalizescores(inboundlinkscore)
    
    
    #使用PageRank分值
    def pagerankscore(self , rows):
        pageranks = dict([(row[0] , self.con.execute("select score from pagerank where urlid=%d" % row[0]).fetchone()[0])
                        for row in rows])
        maxrank = max(pageranks.values())
        
        normalizescores = dict([(u , float(l)/maxrank) for (u , l) in pageranks.items()])
        
        return normalizescores
    
    #使用网页链接显示的那个文本
    #利用链接文本
    def linktextscore(self , rows , wordids):
        linkscores = dict([(row[0] , 0) for row in rows])
        for wordid in wordids:
            cur = self.con.execute('select link.fromid,link.toid from linkwords,link where \
            wordid = %d and linkwords.linkid = link.rowid' % wordid)
            
            for (fromid , toid) in cur:
                if toid in linkscores:
                    pr = self.con.execute('select score from pagerank where urlid=%d' % fromid).fetchone()[0]
                    linkscores[toid] += pr
                
        maxscore = max(linkscores.values())
        
        normalizescores  = dict([(u , float(l)/maxscore) for (u , l) in linkscores.items()])
        
        return normalizescores

In [51]:
e = searcher('searchindex.db')
#e.getmatchrows('functional programming')

e.query('computer games')

OperationalError: no such column: linkwords.linkid

### pagerank